In [1]:
'''
   This program is for SegNet.
'''

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes

import os, sys
import numpy as np
import math
from datetime import datetime
import time
from PIL import Image
from math import ceil
from tensorflow.python.ops import gen_nn_ops
# modules
#from Utils import _variable_with_weight_decay, _variable_on_cpu, _add_loss_summaries, _activation_summary, print_hist_summery, get_hist, per_class_acc, writeImage
#from Inputs import *

'''-------'''
# parma presetting
IMAGE_HEIGHT = 360
IMAGE_WIDTH = 480
IMAGE_DEPTH = 3

'''------------------------------------------------------------------'''
##input

def get_filename_list(filename):
	# 拿到数据集
	fd =open(filename)
	for line in fd:
		line=line.strip().split(" ")
		image_path=line[0]
		label_path=line[1]
	
	return image_path,label_path

def CamVidInput(inputdatafilename,inputlabelfilename,batch_size):
    """
    inputdatafilename: 输入data
    inputlabelfilename: 输入y
    batch_size: 一次计算的量
    读到文件名队列中,然后读出图片.reshape
    """
    images=tf.convert_to_tensor(inputdatafilename,dtype=dtypes.string)
    labels=tf.convert_to_tensor(inputlabelfilename,dtype=dtypes.string)
    
    filename_queue=tf.train.slice_input_producer([images,labels],shuffle=True)
    image_val=tf.read_file(filename_queue[0])
    label_val=tf.read_file(filename_queue[1])
    
    image_bytes=tf.image.decode_png(image_val)
    label_bytes=tf.image.decode_png(label_val)
    
    # 把读到的png图片做一个reshape
    image=tf.reshape(image_bytes,(IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_DEPTH))
    label=tf.reshape(label_bytes,(IMAGE_HEIGHT,IMAGE_WIDTH,1))
    
    return image,label
    


'''------------------------------------------------------------------'''

def training(trainfilepath,valfilepath,batch_size,image_width,image_height,image_ch,max_steps):
	train_image_filenames,train_label_filenames=get_filename_list(trainfilepath)
	val_image_filenames,val_label_filenames=get_filename_list(valfilepath)
	
    startstep=0 # 如果是finetune的话不为0.
    with tf.Graph().as_default():
        # train_data_node和train_label_node
        #    这两者作为训练集的data和label.
        train_data_node=tf.placeholder(tf.float32,shape=[batch_size,image_height,image_width,image_ch])
        train_label_node=tf.placeholder(tf.float32,shape=[batch_size,image_height,image_width,1]) # 它是1个通道.

        #phase_train
        # phase_train作为conv*的输入.是一个True和false的
        phase_train=tf.placeholder(tf.float32,name="phase_train") # 为什么没有设置shape=[]
        global_step=tf.variable(0,trainable=False) # 设置步长,不参与训练
        
        
        # 读出camVid
        train_images,train_labels=CamVidInput(train_image_filenames,train_label_filenames,batch_size)
        val_images,val_labels=CamVidInput(val_image_filenames,val_label_filenames,batch_size)
        
        
        
        # 建立encoder+decoder的网络图
        #     输入: data和y.
        #     返回损失和预测精度
        #     phase_train作用,是Bool型
        loss,eval_prediction=inference(train_data_node,train_label_node,batch_size,phase_train)
        
        
        
        # 建立train的图
        train_op=train(loss,global_step)
        
        with tf.Session() as sess:
            init=tf.global_variables_initializer()
            sess.run(init)
            
            # 创建线程,并用coordinator()管理
            coord=tf.train.Coordinator()
            threads=tf.train.start_queue_runners(sess=sess,coord=coord)
            
            
            for step in range(startstep,startstep+max_steps):
                image_batch,label_batch=sess.run([train_images,train_labels])
                feed_dict={
                    train_data_node:image_batch,
                    train_label_node:label_batch,
                    phase_train:True
                }
                _,loss_value=sess.run([train_op,loss],feed_dict=feed_dict) # 第一个_是不关心的.
                if step%10==0:
                    print("setp:%d,loss=%.2f" %(step,loss_value))
                pred=sess.run(eval_prediction,feed_dict=feed_dict)
                print("pred:%s"pred)
    
    coord.request_stop()
    coord.join(threads)
                
                
                
                
            



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 42)